# Using Jamba on Amazon Bedrock for Analyzing Financial Documents 
Jamba can be used to analyze financial documents. In this notebook, we will first load the Amazon's 10K filing (downloaded from [here](https://d18rn0p25nwr6d.cloudfront.net) and can then ask questions about it. This report has about 40,000 words.

In [11]:
# Required imports
import os
import json
import requests
import time

import boto3


In [12]:
# Function to read the contents of the files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Paths to the text files
document = '10k.txt'

document_content=read_file(document)

In [13]:
def _call_bedrock_jamba(prompt,**kwargs):
    bedrock = boto3.client(service_name="bedrock-runtime")
    
    body={
        "messages":[{"role":"user","content":prompt}],
        "max_tokens": 1024,
        "top_p": 0.8,
        "temperature": 0.7,
    }
    
    body.update(kwargs)
    body = json.dumps(body)
    
    modelId = "ai21.jamba-1-5-large-v1:0"
    
    accept = "application/json"
    contentType = "application/json"
    
    response = bedrock.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )

    print("response: ", response)
    
    result=json.loads(response.get('body').read())
    return result['choices'][0]['message']['content']

In [14]:

def call_jamba(prompt, **kwargs):
    attempts = 0
    while attempts < 5:
        try:
            return _call_bedrock_jamba(prompt, **kwargs)
        except Exception as e:
            print(e)
            attempts += 1
            print(f"Attempt {attempts}: Failed to call API, retrying in 3 seconds...")
            time.sleep(3)
    raise Exception("Failed to complete the API call after 5 attempts")

## Ask a Question
Next we will ask a question from the document, passing the entire content of the document to the prompt. 

In [15]:
question="What was Amazon's revenue generating activity in 2019?"

In [16]:

q_a_prompt = f"""
You are an excellent research assistant. Based the following "Document Content" do your best to answer the question posed.
Keep your answer strictly grounded in the document, and if the answer cannot be found in the document, just say "I do not know"

Document Content:

{document_content}

Question: 
{question}

Answer to Question:
"""


In [ ]:
answer = call_jamba(q_a_prompt,temperature=.7)

# Print the result
print("Answer:")
print(answer)